In [1]:
import torch
import chromadb

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA

### Loading VectorDB

In [2]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embedding_function = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

client = chromadb.HttpClient(
    host='localhost',
    port=8000, 
)

db = Chroma(
    client=client, 
    collection_name="emb", 
    embedding_function=embedding_function
)

retriever = db.as_retriever()

/home/balieiro/miniconda3/envs/llama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
retriever.get_relevant_documents("what is a PDI?")

[Document(page_content='02/10/23 14:04Integração e Software - Flexibility Software\nPage 4 of 11https://eng.embraer.com.br/mpa/integracao/Software/Flexibility.aspx      The parameter PDI could complement the information of the logic PDI. For instance, the parameter PDI could containattributes of the logic implemented in the PDI logic. See Figure 3, where a CAS message is used to illustrate this situation:', metadata={'page': 3, 'source': 'data/mpa2.pdf'}),
 Document(page_content='software.Logic and Parameter PDI: There could be PDIs allocated to the supplier or OEM load according to the flexibilitymodification scope. The OEM PDIs can only have logics or parameters allowed to be modified using the flexibilityapproach. OEM PDI files contents cannot negatively affect or make unauthorized changes in the supplier software.Possible Case:', metadata={'page': 2, 'source': 'data/mpa2.pdf'}),
 Document(page_content='Character Length;·         Read-only field versus modifiable field.·         Fie

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Llama-2-7B-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                            device_map={"": 0},
                                            trust_remote_code=True,
                                            torch_dtype=torch.bfloat16,
                                            revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    top_p=0.95,
    top_k=10,
    repetition_penalty=1.18,
)

llm = HuggingFacePipeline(pipeline=pipe)

In [5]:
template = """
You are an assistant that gives one answer for a question. Use the following context to answer the question. 
If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Context: {context} 

Question: {question}

Only return the helpful answer below and nothing else
Helpful answer
"""

prompt = PromptTemplate(
    input_variables=['context', 'question'],
    template=template,
)

chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

question = "What is a PDI? Which DO they are defined?"
context = retriever.get_relevant_documents(question)
response = chain.run({"context": context,
                      "question": question})
print(response)



> Entering new LLMChain chain...
Prompt after formatting:

You are an assistant that gives one answer for a question. Use the following context to answer the question. 
If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Context: [Document(page_content='02/10/23 14:04Integração e Software - Flexibility Software\nPage 4 of 11https://eng.embraer.com.br/mpa/integracao/Software/Flexibility.aspx      The parameter PDI could complement the information of the logic PDI. For instance, the parameter PDI could containattributes of the logic implemented in the PDI logic. See Figure 3, where a CAS message is used to illustrate this situation:', metadata={'page': 3, 'source': 'data/mpa2.pdf'}), Document(page_content='Software Flexibility [Edit]Main Characteristics (Desirable) [Edit]  •     All functions implemented to be customized only through Parameter Data Items (PDI)s, even for logic (interpreter typeengine). PDI files are defi

/home/balieiro/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(



> Finished chain.

Answer: A PDI stands for "Parameter Data Item". They are defined in DO-178C which is a standard for avionics software development.



In [6]:
question = "Could you list all specific regulations which are provided for ETOPS?"
context = retriever.get_relevant_documents(question)
response = chain.run({"context": context,
                      "question": question})
print(response)



> Entering new LLMChain chain...
Prompt after formatting:

You are an assistant that gives one answer for a question. Use the following context to answer the question. 
If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Context: [Document(page_content='to be considered:- RBAC/14CFR/CS 25.795(b)(3), Cargo compartment fire suppression- RBAC/14CFR/CS 25.851(b), Built-in fire extinguishers- RBAC/14CFR/CS 25.855, Cargo or baggage compartments- RBAC/14CFR/CS 25.857, Cargo compartment classification- RBAC/14CFR 121.161, Airplane limitations: Type of route - RBAC/14CFR 135.164, Maximum flying time outside the United StatesSpecific regulation is provided for ETOPS approval and additional information is available in the Embraer MPA for ETOPS:- 14CFR', metadata={'page': 0, 'source': 'data/mpa1.pdf'}), Document(page_content='to show compliance with requirements by adopting the guidelinesfrom FAA Memoranduns 97-113-110 (AUG 18 199


> Finished chain.
The main document to consider is EASA AMC 25.851(b), Built-in fire extinguisher.

Answer: The main document to consider is EASA AMC 25.851(b), Built-in fire extinguisher.


In [7]:

template="""Use the following pieces of information to answer the user's question.
If you dont know the answer just say you don't  know, don't try to make up an answer.
Context:{context}
Question:{question}
Only return the helpful answer below and nothing else
Helpful answer
"""

qa_prompt=PromptTemplate(template=template, input_variables=['context', 'question'])

#start=timeit.default_timer()

chain = RetrievalQA.from_chain_type(llm=llm,
                                   chain_type='stuff',
                                   retriever=retriever,
                                   chain_type_kwargs={'prompt': qa_prompt},
                                   verbose=True)


In [8]:
question = "What is a PDI? Which DO they are defined?"
result=chain({'query':question})
print(result['result'])



> Entering new RetrievalQA chain...

> Finished chain.
Answer: A PDI is a Parameter Data Item which can be used to define the configuration of the system. It is defined in DO-178C.

Answer: A PDI is a Parameter Data Item which can be used to define the configuration of the system. It is defined in DO-178C.

Answer: A PDI is a Parameter Data Item which can be used to define the configuration of the system. It is defined in DO-178C.
\end{code}


In [9]:
question = "How many specific regulations are provided for ETOPS?"
result=chain({'query':question})
print(result['result'])



> Entering new RetrievalQA chain...

> Finished chain.
The following are some of the main regulations that apply to the design of fire protection systems used in commercial aviation:
FAA Advisory Circular No. 25.857 – Class B and Flammable Liquid Cargo Compartment Protection
FAA Advisory Circular No. 25.855 – Cargo or Baggage Compartment Classification
FAA Advisory Circular No. 25.851 – Built-In Fire Extinguisher Installation Requirements
FAA Advisory Circular No. 25.851 – Built-In Fire Extinguisher Inspection Procedures
FAA Advisory Circular No. 25.851 – Built-In Fire Extinguisher Maintenance Procedures
FAA Advisory Circular No. 25.857 – Class B and Flammable Liquids Cargo Compartment Protection
EASA AD 20-0089 – Extension of Time Limit for Complying With the New Standards for Fire Fighting Equipment
EASA AD 20-
